In [14]:
%load_ext autoreload
%autoreload 2

import os; print(os.getcwd())
import socket; print(socket.gethostname())
import sys; sys.path.insert(0,'/home/ajasja/projects/truncator')
from glob import glob
import Bio
from Bio import SeqIO
import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/home/rdkibler/projects/tiara_gen2/toroids/02_4_churro-9x_xtreme/06_order_2_homotrimer/order
dig112


In [15]:
import truncator

out_dir = "out/R3/!to_order/2018.03.01"
out_dir_pdbs = out_dir+"/pdbs"
truncator.make_dirs(out_dir_pdbs)
pdbs = glob("out/R3/!final/*.pdb")
pdbs = [pdb for pdb in pdbs if "__" in os.path.basename(pdb)]
for pdb in pdbs:
    !cp -v {pdb} {out_dir_pdbs}

In [16]:
excel_name = '200214_C3_symmetric_actors.xlsx'
info = pd.read_excel(excel_name, sheet_name='equivalence')
info = info.fillna('')
if not 'seq' in info:
    info['seq']=''
info['seq_with_tags']='';info['full_seq']=''

all_vectors = set()
for vectors in list(info.vector.unique()):
    for vector in vectors.split(','):
        all_vectors.add(vector)
all_vectors

{'pet29b+'}

In [18]:
truncator.silent_remove('ALL_SEQ.AA.fasta')
truncator.make_dirs('03_fasta_with_TAGS')


out_dir_pdbs = '00_long_name_pdbs/'
pdbs = glob(out_dir_pdbs+"/*.pdb")


for prot in info.itertuples():
    
    if not prot.process in 'yes seq'.split() : continue
    
    print(prot.name)
    if prot.process != "seq":
        truncator.extract_chain(f'{out_dir_pdbs}/{prot.name}.pdb',             chain=prot.chain,  out_dir='01_pdbs',     out_name=prot.short_name+"_"+prot.chain, out_format='.pdb')
        truncator.extract_chain(f'01_pdbs/{prot.short_name}_{prot.chain}.pdb', chain=prot.chain,  out_dir='02_fasta_AA', out_name=prot.short_name+"_"+prot.chain, out_format='.fasta')
    
        #Make fasta with tags
        seq = str(SeqIO.read(f'02_fasta_AA/{prot.short_name}_{prot.chain}.fasta', "fasta").seq)
    else:
        seq = prot.seq
        
    seq_with_tags = prot.N_tag + seq + prot.C_tag
    print(seq)
    seq_with_tags = truncator.remove_whitespace(seq_with_tags)

    
    info.at[prot.Index, 'seq'] = seq
    info.at[prot.Index, 'seq_with_tags'] = seq_with_tags
    #info.at[prot.Index, 'full_seq'] = full_seq
    
    
    with open(f'03_fasta_with_TAGS/{prot.short_name}_{prot.chain}.fasta','w+') as f:
        f.write(f">{prot.short_name}_{prot.chain}\n")
        f.write(seq_with_tags+'\n')
        f.write('\n')

    

actor_9x_017_sym_sub3
 Save: wrote "actor_9x_017_sym_sub3_A.pdb".
PELFLQDLRSLVEAARILARLARQRGDEHALERAARWAEQAARQAEELARQARKEGNLELALKALQILVNAAYVLAEIARDRGNEELLEKAARLAEEAARQAEEIWRQAEKEGNQQLRTKAAHILLRAAEVLLEIARDRGNQELLEKAQRLVDEVARRQQEAARQIRKEGK
actor_9x_018_sym_sub3
 Save: wrote "actor_9x_018_sym_sub3_A.pdb".
PRLVLRALENMVRAAHTLAEIARDNGNEEWLERAARLAEEVARRAEELAREARKEGNLELALKALQILVNAAYVLAEIARDRGNEELLEKAARLAEEAARQAEEIARQARKEGNFELALEALEILNEAARVLARIAHHRGNQELLEKAWRLTHRSAKWSREIAEQARKEGE
actor_9x_019_sym_sub3
 Save: wrote "actor_9x_019_sym_sub3_A.pdb".
PRWYLQHLETLVRAAEILAELARQRGDEEALEEAARMAEHAARQAEELAREARKEGNLELALKALQILVNAAYVLAEIARDRGNEELLEKAARLAEEAARQAKEIEEQAEREGDNQLAIKALRILLEAAHVLMQIARDRGNEELLEKAKRLVREAARRSEERMERQRHEGK
actor_9x_020_sym_sub3
 Save: wrote "actor_9x_020_sym_sub3_A.pdb".
PELALQNLQSIVEAVRVSVRQAKEQGDEELLERVARQAERAARQAEKIAREARKRGNLELALKALQILVNAAYVLAEIARDRGNEELLEKAARLAEEAARQAEEIWEQLRKEGNWQLANKAERILIQAAEVLAEIAKDRGNEELLRKSIELSQRVAERSQEAAEQLQKRGE


In [19]:
#!cat $(ls 02_fasta_AA/*RK05__*B.fasta) > RK05_B.fasta
#CHECK which parts B are the same

In [20]:
in_dir  = '02_fasta_AA'
out_dir_tags = '03_fasta_with_TAGS'
out_dir_full = '04_fasta_full'
out_dir_DNA = '05_fasta_DNA'

In [21]:
#!/home/rdkibler/.conda/envs/domesticator/bin/python /home/rdkibler/projects/domesticator/domesticator.py --create_template --vector pjl1_sfgfp-DOMESTICATOR_INSERT-lg.gb --avoid_kmers_boost 10 
def domesticate(fasta_file, out_name, fake_run=False, parallel=True):
    cmd = f"/home/rdkibler/.conda/envs/domesticator/bin/python /home/rdkibler/projects/domesticator/domesticator.py \
    {fasta_file} \
    --avoid_restriction_sites  XhoI NdeI \
    --avoid_patterns AGGAGG GCTGGTGG ATCTGTT GGRGGT \
    --species e_coli \
    --avoid_kmers 8 \
    --avoid_kmers_boost 10 \
    --output_mode fasta \
    --output_filename {out_name}.DNA.fasta > {out_name}.log "
    if parallel:
        cmd = cmd + '2>&1 &'
    print(cmd)
    if not fake_run:
        !{cmd}
    # --sequence_name {name} \
domesticate('test.fasta', "test", fake_run=True)

/home/rdkibler/.conda/envs/domesticator/bin/python /home/rdkibler/projects/domesticator/domesticator.py     test.fasta     --avoid_restriction_sites  XhoI NdeI     --avoid_patterns AGGAGG GCTGGTGG ATCTGTT GGRGGT     --species e_coli     --avoid_kmers 8     --avoid_kmers_boost 10     --output_mode fasta     --output_filename test.DNA.fasta > test.log 2>&1 &


GROUP_SIZE=1
task_list=tasks_name
from math import ceil; num_tasks = ceil(len(cmds)/GROUP_SIZE)
!export GROUP_SIZE={GROUP_SIZE}; sbatch --mem=4GB -p medium -J {task_list} -o {task_list}.out -e {task_list}.err -a 1-{num_tasks} /home/ajasja/scripts/wrapper_digs_array_job_group.sh {task_list} 

In [22]:
!pwd

/mnt/home/rdkibler/projects/tiara_gen2/toroids/02_4_churro-9x_xtreme/06_order_2_homotrimer/order


In [23]:
truncator.make_dirs("05_fasta_DNA")
print("cd "+os.getcwd())
SKIP_EXISTING = False
for fasta in sorted(glob(f"03_fasta_with_TAGS/*.fasta")):
    #print(basename)
    basename = truncator.basename_noext(fasta)
    if not (SKIP_EXISTING and os.path.exists(f"05_fasta_DNA/{basename}.DNA.fasta") ):
        domesticate(fasta, f"05_fasta_DNA/{basename}", fake_run=True, parallel=True)

cd /mnt/home/rdkibler/projects/tiara_gen2/toroids/02_4_churro-9x_xtreme/06_order_2_homotrimer/order
/home/rdkibler/.conda/envs/domesticator/bin/python /home/rdkibler/projects/domesticator/domesticator.py     03_fasta_with_TAGS/actor_9x_017_sym_sub3_A.fasta     --avoid_restriction_sites  XhoI NdeI     --avoid_patterns AGGAGG GCTGGTGG ATCTGTT GGRGGT     --species e_coli     --avoid_kmers 8     --avoid_kmers_boost 10     --output_mode fasta     --output_filename 05_fasta_DNA/actor_9x_017_sym_sub3_A.DNA.fasta > 05_fasta_DNA/actor_9x_017_sym_sub3_A.log 2>&1 &
/home/rdkibler/.conda/envs/domesticator/bin/python /home/rdkibler/projects/domesticator/domesticator.py     03_fasta_with_TAGS/actor_9x_018_sym_sub3_A.fasta     --avoid_restriction_sites  XhoI NdeI     --avoid_patterns AGGAGG GCTGGTGG ATCTGTT GGRGGT     --species e_coli     --avoid_kmers 8     --avoid_kmers_boost 10     --output_mode fasta     --output_filename 05_fasta_DNA/actor_9x_018_sym_sub3_A.DNA.fasta > 05_fasta_DNA/actor_9x_018_

In [24]:
#confirm transcription
all_correct = True 
for fasta in glob(f"{out_dir_tags}/*.fasta"):
    basename = truncator.basename_noext(fasta)
    DNA_fasta = f"{out_dir_DNA}/{basename}.DNA.fasta"
    #original desired sequence
    orig = str(SeqIO.read(fasta, "fasta").seq)
    #back reversed
    rev = str(SeqIO.read(DNA_fasta, "fasta").translate(to_stop=False).seq)
    if not (orig == rev):
        all_correct = False;
        print(fasta + " DID NOT GET TRANSLATED CORRECTLY")
        print(repr(orig))
        print(repr(rev))
        import difflib
        for i,s in enumerate(difflib.ndiff(orig, rev)):
                if s[0]==' ': continue
                elif s[0]=='-':
                    print(u'Delete "{}" from position {}'.format(s[-1],i))
                elif s[0]=='+':
                    print(u'Add "{}" to position {}'.format(s[-1],i)) 
        print("")
if all_correct:
    print("ALL CORRECT!")
        


ALL CORRECT!


In [25]:
#Make vector gbs and a fasta file with all the sequences
truncator.silent_remove('ALL_SEQ.AA.fasta')
with open('ALL_SEQ.AA.fasta' ,'a') as mf:
    truncator.make_dirs('06_DNA_vectors')
    for prot in info.itertuples():
        if not prot.process in 'yes seq'.split() : continue
        print(f'{prot.short_name}_{prot.chain}')
        insert = SeqIO.read(f'05_fasta_DNA/{prot.short_name}_{prot.chain}.DNA.fasta', "fasta")
        for vector in prot.vector.split(','):
            vector = vector.strip()
            print(f'>{prot.short_name}_{prot.chain}__{vector}', file=mf)
            vector_seq =  Bio.SeqIO.read(f'vectors/{vector}.gb', "genbank")
            vector_seq, loc = truncator.insert_into_vector(vector_seq, 'DOMESTICATOR_INSERT', insert)
            vector_seq.seq.alphabet=Bio.Alphabet.DNAAlphabet()
            gp = truncator.find_annotation(vector_seq, 'GENE_PRODUCT')
            full_sequence_AA = gp.extract(vector_seq).seq.translate(cds=True)
            info.at[prot.Index, 'full_seq'] = str(full_sequence_AA)
            print(str(full_sequence_AA), file=mf)
            vector_out_name = f'06_DNA_vectors/{prot.short_name}_{prot.chain}__{vector}.gb'
            Bio.SeqIO.write(vector_seq, vector_out_name, 'genbank')
        
    

actor_9x_017_sym_sub3_A


TranslationError: Extra in frame stop codon found.

In [35]:
df = truncator.fasta_to_dataframe('ALL_SEQ.AA.fasta')
display(df)
df.to_excel('ALL_SEQ.info.xlsx')

,name,sequence,molecular_weight,molecular_weight_monoisotopic,extinction_280nm_reduced,extinction_280nm_cys_cys,Abs_1mg_ml_280nm_reduced,Abs_1mg_ml_280nm_cys_cys,isoelectric_point,charge_pH7.5,gravy
0,LHD29_A__sfGFP--sm_pJL1-smBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,43210.5206,43184.581701,46870,46995,1.084690,1.087582,5.993835,-8.937728,-0.671611
1,LHD29_A__sfGFP--lg_pJL1-lgBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,59600.0804,59564.017019,65320,65445,1.095972,1.098069,5.603455,-16.834341,-0.460668
2,LHD29_B__sfGFP--sm_pJL1-smBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,42950.0848,42924.321412,41370,41495,0.963211,0.966121,5.483215,-13.087615,-0.629923
3,LHD29_B__sfGFP--lg_pJL1-lgBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,59339.6446,59303.756730,59820,59945,1.008095,1.010202,5.232117,-20.984227,-0.430427
4,LHD37_A__sfGFP--sm_pJL1-smBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,43200.4428,43174.535813,35870,35995,0.830316,0.833209,6.039612,-7.974699,-0.680307
5,LHD37_A__sfGFP--lg_pJL1-lgBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,59590.0026,59553.971131,54320,54445,0.911562,0.913660,5.623474,-15.871312,-0.466976
6,LHD37_B__sfGFP--sm_pJL1-smBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,43047.1621,43021.381387,35870,35995,0.833272,0.836176,5.495544,-13.086417,-0.671867
7,LHD37_B__sfGFP--lg_pJL1-lgBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,59436.7219,59400.816705,54320,54445,0.913913,0.916016,5.245178,-20.983030,-0.460853
8,LHD96_A__sfGFP--sm_pJL1-smBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,43715.2208,43688.910225,32890,33015,0.752370,0.755229,5.992737,-8.010888,-0.597756
9,LHD96_A__sfGFP--lg_pJL1-lgBiT_SL,MEKKIWSHPQFEKGGSSKGEELFTGVVPILVELDGDVNGHKFSVRG...,60104.7806,60068.345543,51340,51465,0.854175,0.856255,5.574158,-15.907500,-0.410565
